# Database Helper
Database connection and query utilities.
Run this with: `%run /home/jovyan/work/system/helpers/01_Database.ipynb`

In [ ]:
# Load configuration if not already loaded
if 'DB_CONFIG' not in globals():
    %run /home/jovyan/work/system/helpers/00_Config.ipynb

In [ ]:
def get_connection():
    """Create and return a database connection."""
    conn_str = (
        f"DRIVER={{ODBC Driver 18 for SQL Server}};"
        f"SERVER={DB_CONFIG['server']},{DB_CONFIG['port']};"
        f"DATABASE={DB_CONFIG['database']};"
        f"UID={DB_CONFIG['username']};"
        f"PWD={DB_CONFIG['password']};"
        f"TrustServerCertificate=yes;"
    )
    return pyodbc.connect(conn_str)

def execute_query(query: str, params: tuple = None) -> pd.DataFrame:
    """Execute a SELECT query and return results as DataFrame."""
    with get_connection() as conn:
        return pd.read_sql_query(query, conn, params=params)

def execute_command(query: str, params: tuple = None) -> int:
    """Execute an INSERT/UPDATE/DELETE command and return rows affected."""
    with get_connection() as conn:
        cursor = conn.cursor()
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        conn.commit()
        return cursor.rowcount

def execute_scalar(query: str, params: tuple = None) -> Any:
    """Execute a query and return a single value."""
    with get_connection() as conn:
        cursor = conn.cursor()
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        result = cursor.fetchone()
        return result[0] if result else None

def test_connection() -> bool:
    """Test database connection."""
    try:
        result = execute_scalar("SELECT 1")
        return result == 1
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        return False

def get_active_cycle() -> Optional[Dict[str, Any]]:
    """Get the active cycle information."""
    query = """
    SELECT c.*, l.locked_by, l.locked_at
    FROM irp_cycle c
    INNER JOIN irp_system_lock l ON l.active_cycle_id = c.id
    WHERE c.status = 'active'
    """
    df = execute_query(query)
    if not df.empty:
        return df.iloc[0].to_dict()
    return None

def init_database() -> bool:
    """Initialize database from SQL script."""
    sql_file = SYSTEM_PATH / 'db' / 'init_database.sql'
    if not sql_file.exists():
        print(f"❌ Database init script not found: {sql_file}")
        return False
    
    try:
        with get_connection() as conn:
            cursor = conn.cursor()
            
            # Read and execute the SQL script
            with open(sql_file, 'r') as f:
                sql_script = f.read()
            
            # Split by GO statements and execute each batch
            for batch in sql_script.split('\nGO\n'):
                if batch.strip():
                    cursor.execute(batch)
            
            conn.commit()
            print("✅ Database initialized successfully")
            return True
    except Exception as e:
        print(f"❌ Database initialization failed: {e}")
        return False

# Test connection on load
if test_connection():
    print("✅ Database connection successful")
else:
    print("⚠️  Database connection failed - you may need to initialize the database")